# Exploratory Data Analysis on Job Reviews

In [39]:
import pandas as pd
import cufflinks as cf
from textblob import TextBlob
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Reading the data

In [40]:
df = pd.read_csv("review.csv") 
df

,Unnamed: 0,id,company,title,rating,author,author_status,location,date,description,source
0,0,1,bmo financial group,Decent,3.0,Credit Analyst (Current Employee),NaN,"Burnaby, BC",11 July 2018,Not a bad job but can get boring easily. Manag...,indeed.com
1,1,2,bmo financial group,easy place to work,3.0,Project Manager (Former Employee),NaN,"Toronto, ON",7 April 2020,co-workers are friendly but pay is not good at...,indeed.com
2,2,3,bmo financial group,Dead Industry- They will squeeze you like a le...,1.0,Assistant Manager (Former Employee),NaN,"Cambridge, ON",6 April 2020,They give you the illusion of how great of a j...,indeed.com
3,3,4,bmo financial group,"In BMO agile team, one developer has to servic...",2.0,Software Specialist (Former Employee),NaN,"Toronto, ON",6 April 2020,"Basically, in BMO agile team, they hire 5+ per...",indeed.com
4,4,5,bmo financial group,Good,5.0,Relationship Manager (Current Employee),NaN,"Nanaimo, BC",4 April 2020,Work/life balance and customer centric. Traini...,indeed.com
...,...,...,...,...,...,...,...,...,...,...,...
790,790,791,amazon,You have an hourly quota....if not met you get...,2.0,Order Picker/Forklift Operator (Former Employee),NaN,"Ottawa, ON",18 January 2020,Very fast paced hard on your feet need very co...,indeed.com
791,791,792,amazon,Most worst job I ever worked at,1.0,Hard worker (Current Employee),NaN,"Brampton, ON",18 January 2020,Alotta bad stuff happened i would report it an...,indeed.com
792,792,793,amazon,It is a good company,4.0,Risk Analyst • Fraud Risk & Compliance (Former...,NaN,"Barrie, ON",18 January 2020,It is a good company . I would recommend peopl...,indeed.com
793,793,794,amazon,competitive but fun,4.0,seasonal warehouse associate (Former Employee),NaN,"Brampton, ON",17 January 2020,it is a faced paced work environment and compe...,indeed.com


# Data Preprocessing

In [41]:
df = df[~df['description'].isnull()]

def preprocess(DescriptionText):
    DescriptionText = DescriptionText.str.replace("(<br/>)", "")
    DescriptionText = DescriptionText.str.replace('(<a).*(>).*(</a>)', '')
    DescriptionText = DescriptionText.str.replace('(&amp)', '')
    DescriptionText = DescriptionText.str.replace('(&gt)', '')
    DescriptionText = DescriptionText.str.replace('(&lt)', '')
    DescriptionText = DescriptionText.str.replace('(\xa0)', ' ')  
    return DescriptionText
df['description'] = preprocess(df['description'])

In [42]:
df['review_len'] = df['description'].astype(str).apply(len)
df['word_count'] = df['description'].apply(lambda x: len(str(x).split()))

In [43]:
#Distribution of Review Ratings
df['rating'].iplot(
    kind='hist',
    xTitle='rating',
    linecolor='black',
    yTitle='count',
    color ='Purple',
    title='Review Rating Distribution')

In [44]:
#The distribution review text lengths
df['review_len'].iplot(
    kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    color ='Cyan',
    title='Review Text Length Distribution')

In [45]:
# The distribution of review word count
df['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='word count',
    linecolor='black',
    yTitle='count',
    color = 'Grey',
    title='Review Text Word Count Distribution')

In [46]:
# top 20 words in description before removing stop keyword
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(corpus, n=None):
 
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]



common_words = get_top_n_words(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Red', title='Top 20 words in job-description before removing stop words')

In [47]:
# top 20 words in description after removing stop keyword
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Green', title='Top 20 words in job-description after removing stop words')

In [48]:
#The distribution of top bigrams before removing stop words
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df3 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df3.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Red', title='Top 20 bigrams in the job-description before removing stop words')

In [49]:
#The distribution of top bigrams after removing stop words
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', color='Green',title='Top 20 bigrams in the job-description after removing stop words')

In [50]:
#The distribution of Top trigrams before removing stop words

def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df5 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df5.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black', color='Red',title='Top 20 trigrams in the job-description before removing stop words')


In [51]:
#The distribution of Top trigrams after removing stop words

def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['description'], 20)
# for word, freq in common_words:
#     print(word, freq)
df6 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df6.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='Green', title='Top 20 trigrams in the job-description after removing stop words')